### Setup the Model
Note: need to install following dependencies:
pipenv install gym
pipenv install pyglet
pipenv install pyopengl
pipenv install -qq pyvirtualdisplay
pipenv install tensorflow
pipenv install torch  <- not sure if I need

In [1]:
import os
import gym
from stable_baselines3  import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
environment_name = 'CartPole-v0'
env = gym.make(environment_name)

In [3]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode {}  Score: {}'.format(episode, score))
env.close()

Episode 1  Score: 11.0
Episode 2  Score: 16.0
Episode 3  Score: 10.0
Episode 4  Score: 22.0
Episode 5  Score: 10.0


In [32]:
env.close()

In [4]:
env.action_space

Discrete(2)

In [5]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

### Train the ML Model


In [6]:
# Make directories first. 'Logs' is a subfolder of training
log_path = os.path.join('Training','Logs')
log_path

'Training/Logs'

In [7]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [9]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 2286 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1611         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0034567006 |
|    clip_fraction        | 0.0196       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.565       |
|    explained_variance   | -0.0129      |
|    learning_rate        | 0.0003       |
|    loss                 | 0.374        |
|    n_updates            | 110          |
|    policy_gradient_loss | -0.00203     |
|    value_loss           | 6.87         |
----------------------------

### Save and Reload Model

In [10]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')
model.save(PPO_Path)

In [11]:
del model
model = PPO.load(PPO_Path, env=env)

In [12]:
model.learn(total_timesteps=1000)

Logging to Training/Logs/PPO_3
-----------------------------
| time/              |      |
|    fps             | 2520 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------


### Test & Evaluate our model

In [13]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/Users/scottsullivan/.local/share/virtualenvs/python-machinelearning-LC9u9s6X-python/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [17]:
obs = env.reset()
action, _ = model.predict(obs)
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)   # NOW USING MODEL
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode {}  Score: {}'.format(episode, score))
# env.close()

Episode 1  Score: [200.]
Episode 2  Score: [200.]
Episode 3  Score: [200.]
Episode 4  Score: [200.]
Episode 5  Score: [200.]


In [18]:
env.close()